## Introduction

Biological systems are rarely linear, so nonlinear models can capture complex sequence-to-function relationships more effectively. In this notebook, we explore random forests (RF), support vector regression (SVR), and multi-layer perceptrons (MLP) for predicting functional properties from sequences. These models balance predictive power with interpretability, enabling us to model intricate biological patterns beyond what linear models can capture.

The data we are using for the sequence-to-fitness prediction is from ProteinGym (https://proteingym.org), a collection of benchmarks aiming at comparing the ability of models to predict the effects of protein mutations. We first import essential libraries, and data from the fitness folder.

We use scikit-learn, a nice package for building and evaluating machine learning models.

In [1]:
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

In [ ]:
CAPSD = pd.read_csv("data_fitness/CAPSD_AAV2S_Sinai_2021.csv")
PHOT = pd.read_csv("data_fitness/PHOT_CHLRE_Chen_2023.csv")
POLG = pd.read_csv("data_fitness/POLG_DEN26_Suphatrakul_2023.csv")

We first process the data from CAPSD dataset, doing one-hot encoding for the amino acid sequences and then the train-test split. We take the first 5000 entries to keep runtime manageable.

mutated_sequence contains amino acid sequences.

DMS_score is the experimentally measured fitness.

In [ ]:
sequences = CAPSD["mutated_sequence"].values[:5000]
scores = CAPSD["DMS_score"].values[:5000]

In [4]:
amino_acids = list("ACDEFGHIKLMNPQRSTVWY") 
encoder = OneHotEncoder(categories=[amino_acids] * len(sequences[0]))
seq_list = [list(seq) for seq in sequences]
X = encoder.fit_transform(seq_list)
X.shape

(5000, 14700)

We use an 80/20 train–test split. Since fitness values vary in scale, we apply StandardScaler to the labels.

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train_noscale, y_test_noscale = train_test_split(
    X, scores, test_size=0.2, random_state=42)

y_scaler = StandardScaler()
y_train= y_scaler.fit_transform(y_train_noscale.reshape(-1, 1)).ravel()
y_test = y_scaler.transform(y_test_noscale.reshape(-1, 1)).ravel()

X_train.shape, X_test.shape

((4000, 14700), (1000, 14700))

We then define the helper function to print different metrics for evaluating machine learning models on the held-out test set:

-R2 score (coefficient of determination)

-Mean Squared Error (MSE)

-Mean Absolute Error (MAE)

Higher R2 score, lower MSE and MAE indicate a better machine learning model.

In [6]:
def evaluate_model(name, y_true, y_pred):
    print(f"--- {name} ---")
    print("R2:", r2_score(y_true, y_pred))
    print("MSE:", mean_squared_error(y_true, y_pred))
    print("MAE:", mean_absolute_error(y_true, y_pred))
    print()

  #### 1. SVR
Support Vector Regression (RBF Kernel)
SVR with an RBF kernel is powerful but computationally expensive for large feature spaces like one-hot encoded protein sequences.
We try two settings of the regularization parameter C:

For C=1, epsilon=0.1 (the default setting for the SVR in sklearn)

In [15]:
svr = SVR(kernel="rbf", C=1, epsilon=0.1)

svr.fit(X_train, y_train)
y_pred_svr = svr.predict(X_test)

evaluate_model("SVR (RBF kernel)", y_test, y_pred_svr)

--- SVR (RBF kernel) ---
R2: 0.4304178045236362
MSE: 0.5782179751867897
MAE: 0.6125478750517874



The performance is worse than linear model. But if we reduce regularization parameter C to C=0.1, we will get a better performance:

In [14]:
svr = SVR(kernel="rbf", C=10, epsilon=0.1)

svr.fit(X_train, y_train)
y_pred_svr = svr.predict(X_test)

evaluate_model("SVR (RBF kernel)", y_test, y_pred_svr)

--- SVR (RBF kernel) ---
R2: 0.6082557674117688
MSE: 0.39768370352382193
MAE: 0.49260235995353174



#### 2. Multi-Layer Perceptron (Neural Network)
A two‑layer neural network is trained with ReLU nonlinearities.

In [18]:
mlp = MLPRegressor(
    hidden_layer_sizes=(256, 128),
    activation="relu",
    solver="adam",
    max_iter=200,
    random_state=42
)

mlp.fit(X_train, y_train)
y_pred_mlp = mlp.predict(X_test)

evaluate_model("MLP Neural Network", y_test, y_pred_mlp)

--- MLP Neural Network ---
R2: 0.7996177778725667
MSE: 0.20342033803400744
MAE: 0.3395473563043571



#### 3. Random Forest Regression
Random Forests model nonlinear interactions through an ensemble of decision trees.

In [17]:
rf = RandomForestRegressor(
    n_estimators=20,
    max_depth=None,
    random_state=42
)

rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

evaluate_model("Random Forest", y_test, y_pred_rf)

--- Random Forest ---
R2: 0.7234071114426441
MSE: 0.28078648041121296
MAE: 0.3920988405903088



The results show that nonlinear models performs better on the sequence-to-fitness problem compared to linear models. However, parameter selection can affect the model performance a lot. Cross-validation can be used to select better parameters for the model.